# Reflexion Agents in LangGraph

This fixes the challenge that reflection agents have which is that there output is not based on live data.

The reflexion agent although similar to reflection agents fact checks it with external data by making api calls.
In Reflection agent we had to rely on the trained data of the LLM but here we are not limited to that.

# Components of a Reflexion agents

1. The actor. Is the main agent that drives everything it reflects on its responses and it re-executes. It can do this with or without tools to improve on self critique that is grounded in external data.
   Its main sub-components include:
     a. Tools/ tool execution
     b. Initial responder:- Generate an initial response and self reflection
     c. Revisor: reresponds and reflects based on previous reflections 

# Episonic Memory

This refer to the ability of a reflexion agent to recall past interactions, events and experiences based on past interactions.
This is crucial in making agents context aware and human like.
 




# LLM RESPONSE PARSER SYSTEM

This converts unstructure LLM outputs into well-defined python objects. It consists of

1. Chat Prompt template
2. Function calling with pydatic schema: Similar to how we call tools we can also send a schema to the LLM and force it to structure its JSON output according to the schema
3. Pydatic parser: It takes the JSON output from the LLM's function call, Validates and create instances of pydatic classes with the validated data

In [23]:
from pydantic import BaseModel, Field

class Reflexion(BaseModel):
    """
    Reflexion is a model that represents a reflection or thought process.
    It includes an missing,superflous a list of related thoughts.
    """
    missing:str = Field(description="Missing information in the reflexion")
    superflous:str = Field(description="Critique Superfluous information in the reflexion")
    

class AnswerQuestion(BaseModel):
    """
    AnswerQuestion is a model that represents an answer to a question.
    It includes the question, the answer, and a list of related thoughts.
    """
    answer: str = Field(description="250 word detailed answer to the question")
    search_queries: list[str] = Field(description="1-3 queries for researching the question")
    reflexions: list[Reflexion] = Field(
        description="List of reflexions related to the answer"
    )

class ReviseAnswer(AnswerQuestion):
    """
    ReviseAnswer is a model that represents a revised answer to a question.
    """
    refrences: list[str] = Field(description="List of references used in the revised answer")
    


In [24]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
import datetime
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.messages import HumanMessage


llm = ChatOpenAI(model="gpt-4o")
pydatic_tools_parser = PydanticToolsParser(
    tools=[AnswerQuestion],
)
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are expert AI researcher.
Current time: {time}

1. {first_instruction}
2. Reflect and critique your answer. Be severe to maximize improvement.
3. After the reflection, **list 1-3 search queries separately** for researching improvements. Do not include them inside the reflection.
""",
        ),
        MessagesPlaceholder(variable_name="messages"),
        ("system", "Answer the user's question above using the required format."),
    ]
).partial(
    time=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
)

revise_instructions = """Revise your previous answer using the new information.
    - You should use the previous critique to add important information to your answer.
        - You MUST include numerical citations in your revised answer to ensure it can be verified.
        - Add a "References" section to the bottom of your answer (which does not count towards the word limit). In form of:
            - [1] https://example.com
            - [2] https://example.com
    - You should use the previous critique to remove superfluous information from your answer and make SURE it is not more than 250 words.
"""



first_responder_prompt_template = actor_prompt_template.partial(
    first_instruction="Answer the question with a 250 word detailed answer, including any necessary context or background information."
)
first_responder_chain = first_responder_prompt_template | llm.bind_tools([AnswerQuestion], tool_choice="AnswerQuestion")

revisor_chain = actor_prompt_template.partial(
    first_instruction=revise_instructions
) | llm.bind_tools([ReviseAnswer], tool_choice="ReviseAnswer") 

revisor_input = {
    "messages": [
        HumanMessage(
            content="This is the initial answer that needs revision."
        )
    ]
}

# response = first_responder_chain.invoke(
#     {
#         "messages": [
#             HumanMessage(
#                 content="What are the key differences between supervised and unsupervised learning in machine learning?"
#             )
#         ]
#     }
# )

# final_response = revisor_chain.invoke(
#     {
#         "messages": [
#             HumanMessage(
#                 content=response[0]["answer"]
#             )
#         ]
#     }
# )

# print(response)

In [25]:
# Excution tool

import json
from typing import Any, Dict, List, Optional
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage, ToolMessage
from langchain_community.tools import TavilySearchResults

travaly_tool = TavilySearchResults(max_results=3, language="en")

def execute_tool(state: List[BaseMessage]) -> List[BaseMessage]:
    last_ai_message = state[-1]

    if not hasattr(last_ai_message, "tool_calls") or not last_ai_message.tool_calls:
        return []
    tool_messages = []
    for tool_call in last_ai_message.tool_calls:
        if tool_call['name'] in ['AnswerQuestion', 'ReviseAnswer']:
            call_id = tool_call['id']
            search_queries = tool_call["args"].get("search_queries", [])

            query_results={}
            for query in search_queries:
                result = travaly_tool.invoke(query)
                query_results[query] = result
            tool_messages.append(
                ToolMessage(
                    content=json.dumps(query_results),
                    tool_call_id= call_id 
                )
            )
    return tool_messages

            

In [26]:
# reflexion graph

from typing import List
from langchain_core.messages import BaseMessage, ToolMessage
from langgraph.graph import END, MessageGraph

graph = MessageGraph()
MAX_ITERATIONS = 2

graph.add_node("responder",first_responder_chain)
graph.add_node("execute_tools",execute_tool)
graph.add_node("revisor",revisor_chain)

graph.add_edge("responder", "execute_tools")
graph.add_edge("execute_tools", "revisor")

def event_loop(state: List[BaseMessage]) -> str:
    count_tool_visits = sum(isinstance(item, ToolMessage) for item in state)
    num_iterations = count_tool_visits
    if num_iterations > MAX_ITERATIONS:
        return END
    return "execute_tools"

graph.add_conditional_edges("revisor", event_loop)
graph.set_entry_point("responder")

app = graph.compile()
print(app.get_graph().draw_mermaid())
app.get_graph().print_ascii()
response = app.invoke("Write about how small business can leverage AI to grow")
print(response)


---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	responder(responder)
	execute_tools(execute_tools)
	revisor(revisor)
	__end__([<p>__end__</p>]):::last
	__start__ --> responder;
	execute_tools --> revisor;
	responder --> execute_tools;
	revisor --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

  +-----------+    
  | __start__ |    
  +-----------+    
        *          
        *          
        *          
  +-----------+    
  | responder |    
  +-----------+    
        *          
        *          
        *          
+---------------+  
| execute_tools |  
+---------------+  
        *          
        *          
        *          
   +---------+     
   | revisor |     
   +---------+     
        *          
        *          
        *          
   +---------+     
   | __end__ |     
   +---------+     
[HumanMessage(content='Write about how small bu